# 105. Construct Binary Tree from Preorder and Inorder Traversal

Given two integer arrays `preorder` and `inorder` where `preorder` is the preorder traversal of a binary tree and `inorder` is the inorder traversal of the same tree, construct and return the binary tree.

## Example 1:

Input: preorder = [3,9,20,15,7], inorder = [9,3,15,20,7]
Output: [3,9,20,null,null,15,7]

## Example 2:

Input: preorder = [-1], inorder = [-1]
Output: [-1]


## Constraints:

* `1 <= preorder.length <= 3000`
* `inorder.length == preorder.length`
* `-3000 <= preorder[i], inorder[i] <= 3000`
* `preorder` and `inorder` consist of unique values.
* Each value of `inorder` also appears in `preorder`.
* `preorder` is guaranteed to be the preorder traversal of the tree.
* `inorder` is guaranteed to be the inorder traversal of the tree.

In [6]:
from typing import *

In [7]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# Preorder Traversal Algorithm:

In [8]:
def preorder_traversal(root: TreeNode, verbose: bool=False)->List[int]:
    """
    Given a binary tree, return a preorder traversal array of the values.
    :param root:        Root node of the binary tree.
    :param verbose:     If True, prints additional information.
    :return:            Resulting traversal array.
    """
    stack = [root]
    preorder_arr = []
    while len(stack) > 0:
        node = stack.pop()
        # process current node
        if verbose: print(node.val)
        preorder_arr.append(node.val)
        if node.right:
            # push right node to be processed later
            stack.append(node.right)
        if node.left:
            # push left node to be processed next
            stack.append(node.left)
    return preorder_arr

# Inorder Traversal Algorithm

In [9]:
def inorder_traversal(root: TreeNode, verbose: bool=False)->List[int]:
    """
    Given a binary tree, return an inorder traversal array of the values.
    :param root:        Root node of the binary tree.
    :param verbose:     If True, prints additional information.
    :return:            Resulting traversal array.
    """
    stack = [(root, False)] # (node, process)
    # if proces = False, this node is to be traversed
    # if process = True, this node has already been traversed,
    # and it is now time to process it
    inorder_arr = []
    while len(stack) > 0:
        node, process = stack.pop()

        if process:
            # left node has been processed, now process this node
            if verbose: print(node.val)
            inorder_arr.append(node.val)
        else:
            if node.right:
                # push right node to be traversed later
                stack.append((node.right, False))
            # push current node to be processed later but before the right node
            stack.append((node, True))
            if node.left:
                # push the left node to be traversed next
                stack.append((node.left, False))
    return inorder_arr

In [10]:
class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int], verbose: bool=False) -> Optional[TreeNode]:
        """
        Given a preorder and inorder traversal of a tree, return the binary tree representation.
        :param preorder: Array corresponding to the preorder traversal of the binary tree.
        :param inorder:  Array corresponding to the inorder traversal of the binary tree.
        :param verbose:  If True, prints additional information (not implemented in this function).
        :return:         The corresponding binary tree representation.
        """
        n = len(preorder)
        if n == 1:
            # base case
            return TreeNode(preorder[0])

        # Short for "next_unprocessed_idx_preorder". The immediate child of the current node being built
        # must be the very first unprocessed node in the preorder array. We consider a node processed
        # once it has been first created, not once its children have been set.
        nuip = 0

        def _getSplit(val: int, arr_range: Tuple[int, int])->Tuple[Tuple[int, int], Tuple[int, int]]:
            """
            We search for a value in the `inorder` array within some specified range.

            We then return the left index range (arr_range[0], arr_range[0] + idx) where idx is the index of the val
            in inorder[arr_range[0], arr_range[1]]. We must add arr_range[0] to correct the right index.

            Likewise, we also return the right index range (arr_range[0] + idx, arr_range[1]).

            Example: Given array [6, 7, 8, 9, 10] with range (1, 5) and value 8. l_range would be (1, 2), the indices
            of the elements completely left of 8 and within the given range. r_range would be (3, 5), the indices of
            the elements completely right of 8 and within the given range.

            :param val:         Value to search for in inorder[arr_range[0]:arr_range[1]]
            :param arr_range:   Range to search in the inorder array.
            :return l_range:    Range of indices to the left of val
            :return r_range:    Range of indices to the right of val
            """
            # unpack indices of the range. `l_idx` is inclusive, `r_idx` is exclusive
            l_idx, r_idx = arr_range
            # find the value in the subarray in `inorder`
            idx = inorder[l_idx:r_idx].index(val)
            # form the ranges to the left and right of the value
            l_range = (l_idx, l_idx + idx)
            r_range = (l_idx + idx + 1, r_idx)
            return l_range, r_range

        def _formTree(i: int, arr_range: Tuple[int, int])->TreeNode:
            """
            Main recursive function used to build a tree whose root node
            is given as `preorder[i]`. The tree is built using the `preorder` and
            `inorder` arrays as well as a global `nuip` counter to help with tracking
            the next immediate children.
            :param i:           Index into preorder which gives the root node of this tree
            :param arr_range:   This is an index range into the `inorder` array where
                                inorder[arr_range[0], arr_range[1]] are the set of nodes
                                that correspond to the left and right children of this current node.
            :return:
            """
            nonlocal nuip # make nonlocal since it will be modified on each invocation

            # Initialize the node with empty children. It is considered processed once the node
            # is initialized, not later once the node has been formed as a tree. Thus, we
            # increment nuip as well.
            val = preorder[i]
            node = TreeNode(val)
            nuip += 1

            # Parse arr_range where left_range is the subset of nodes strictly to the left
            # of this node, and right_range is the subset of nodes strictly to the right
            # of this node.
            left_range, right_range = _getSplit(val, arr_range)

            if left_range[0] < left_range[1]:
                # non-empty range, we must form the tree of the left child
                node.left = _formTree(nuip, left_range)

            if right_range[0] < right_range[1]:
                # non-empty range, we must form the tree of the right child
                node.right = _formTree(nuip, right_range)

            # this tree has been fully formed, return it
            return node

        # return the root node whose tree is fully formed
        return _formTree(0, (0, n))

def main():
    test_cases = {
        "1": {
            "preorder": [3,9,20,15,7],
            "inorder": [9,3,15,20,7],
        },
        "2": {
            "preorder": [-1],
            "inorder": [-1],
        },
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        # get ground truth arrays
        preorder_gt = targs.get("preorder")
        inorder_gt = targs.get("inorder")
        # build the tree
        ret = solution.buildTree(**targs, verbose=True)
        # get the preorder and inorder arrays from this tree
        preorder_ret = preorder_traversal(ret, verbose=False)
        inorder_ret = inorder_traversal(ret, verbose=False)
        # check that the preorder and inorder array formed from this tree
        # match the ground truth arrays
        passed = (preorder_gt == preorder_ret) and (inorder_gt == inorder_ret)
        # print the results
        print(f"test case {tk}: {targs}\nPassed:{passed}")

main()

test case 1: {'preorder': [3, 9, 20, 15, 7], 'inorder': [9, 3, 15, 20, 7]}
Passed:True
test case 2: {'preorder': [-1], 'inorder': [-1]}
Passed:True


The following is the solution given by Leetcode which has a time and memory complexity of $\mathcal{O}(N)$. Notice how they use a hash-map to get the left and right indices in constant time, whereas you use the `_getSplit` method to search for the value. Besides this, the Leetcode solution and your solution are the exact same.

However, you should notice that there are trade-offs between your solution and the LC solution. Although your solution is slower (due to searching for the idx range instead of using a hash-map), you can handle the case where the values in the binary tree are *not unique*. The solution offered by LC is faster because it returns an idx in constant time, however, *their implementation fails if the values in the binary tree are not unique*.

In [ ]:
class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int]) -> TreeNode:

        def array_to_tree(left, right):
            nonlocal preorder_index
            # if there are no elements to construct the tree
            if left > right:
                return None

            # select the preorder_index element as the root and increment it
            root_value = preorder[preorder_index]
            root = TreeNode(root_value)

            preorder_index += 1

            # build left and right subtree
            # excluding inorder_index_map[root_value] element because it's the root
            root.left = array_to_tree(left, inorder_index_map[root_value] - 1)
            root.right = array_to_tree(inorder_index_map[root_value] + 1, right)

            return root

        preorder_index = 0

        # build a hashmap to store value -> its index relations
        inorder_index_map = {}
        for index, value in enumerate(inorder):
            inorder_index_map[value] = index

        return array_to_tree(0, len(preorder) - 1)